In [2]:
from z3 import *

from itertools import combinations
from z3 import *
from utils import *
import math

In [3]:
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars, name = ""):
    return And([Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)])

def exactly_one_np(bool_vars, name = ""):
    return And(at_least_one_np(bool_vars), at_most_one_np(bool_vars, name))

In [4]:
def at_least_one_seq(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_seq(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    s = [Bool(f"s_{name}_{i}") for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))
    for i in range(1, n - 1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1])))
        constraints.append(Or(Not(s[i-1]), s[i]))
    return And(constraints)

def exactly_one_seq(bool_vars, name):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars, name))

In [5]:
def at_least_k_seq(bool_vars, k, name):
    return at_most_k_seq([Not(var) for var in bool_vars], len(bool_vars)-k, name)

def at_most_k_seq(bool_vars, k, name):
    constraints = []
    n = len(bool_vars)
    s = [[Bool(f"s_{name}_{i}_{j}") for j in range(k)] for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0][0]))
    constraints += [Not(s[0][j]) for j in range(1, k)]
    for i in range(1, n-1):
        constraints.append(Or(Not(bool_vars[i]), s[i][0]))
        constraints.append(Or(Not(s[i-1][0]), s[i][0]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][k-1])))
        for j in range(1, k):
            constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][j-1]), s[i][j]))
            constraints.append(Or(Not(s[i-1][j]), s[i][j]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2][k-1])))   
    return And(constraints)

def exactly_k_seq(bool_vars, k, name):
    return And(at_most_k_seq(bool_vars, k, name+'1'), at_least_k_seq(bool_vars, k, name))

In [21]:
'''n packs
   m couriers
   D matix int(n+1,m+1)
   sz array of item's sizes
   cpt array of couriers' capacities
'''

def numPackCour(numCour, v):
   return Sum(v[numCour])

def listPackCour(numCour, v, pred):
   check = False
   i=-1
   while not check and i<len(v[numCour]):
      i+=1
      If(v[numCour][i]==True, check=True, check=False)
      

      
            

def mcp(n, m, D, sz, cpt):
   
   #pred[i][j] = true if the j-th item is the predecessor of the i-th item
   pred = [[Bool(f"pred({i})_{j}")for j in range(n+1)]for i in range(n+1)]
   #print(pred)

   #create Solver
   solv= Solver()

   col_n = []
   for j in range(n):
      col_n += [pred[j][n]]
   solv.add(exactly_k_seq(pred[n],m, f"PR_{n}")) #pred row
   solv.add(exactly_k_seq(col_n,m, f"PC_{n}")) #pred col
   print(solv.check())


   #i=0 to i=n-1 constraint At most one
   #j=0 to j=n-1 constraint At most one
   for i in range(n):
      col_i = []
      for j in range(n+1):
         col_i += [pred[j][i]]

      solv.add(exactly_one_seq(col_i, f"PC_{i}"))
      print(pred[i]) 
      solv.add(exactly_one_seq(pred[i], f"PR_{i}"))

   



   '''vehicle matrix 
      col = packs
      rows = courriers 
      each rows represent the list of packs foreach courrier
   '''
   #create matrix v mxn
   v=[[Bool(f"vehicle({i})_{j}")for j in range(n)]for i in range(m)]
   

   #each pack have to be carried by exactly one courrier
   for j in range(n):
      solv.add(exactly_one_seq([v[i][j] for i in range(m)], f"v_{j}"))
   
   solv.check()
   mod = solv.model()
   #print(mod)
   #print(mod.evaluate(pred[0][0]))
   return [(i, j) for j in range(n) for i in range(m) if mod.evaluate(v[i][j])]

print(mcp(10,3,0,0,0))



SyntaxError: keyword argument repeated: check (2784752076.py, line 16)

10
